# Class imbalance processing

## SGM

### SMOTE all minority classes to (number of data set samples / classes)

In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
def create_file_if_not_exist(file_folder, file_name):
  if not os.path.exists(file_folder):
    os.mkdir(file_folder)

  file_path = os.path.join(file_folder, file_name)
  if not os.path.exists(file_path):
    with open(file_path, 'x'):
      pass  # Just create the file and do nothing else

In [3]:
#Put all the data together, list1 is the data, list2 is the dichotomy label, and list10 is the 10-category label.
from collections import Counter
import os
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

#Read the tf file data
list1 = []  #data
list2 = []  #label_2
list10 = [] #label_10

for serialized_example in tf.python_io.tf_record_iterator("/kaggle/input/normalized/train_select_12.tfrecords"):
# for serialized_example in tf.python_io.tf_record_iterator("/content/drive/MyDrive/Colab Notebooks/UNSW-NB15 - CSV Files/normalized/train_select_12.tfrecords"):
    example = tf.train.Example()
    example.ParseFromString(serialized_example)

    feature = example.features.feature['features'].float_list.value
    label_2 = example.features.feature['label_2'].float_list.value
    label_10 = example.features.feature['label_10'].float_list.value

    list1.append(feature)
    list2.append(label_2)
    list10.append(label_10)

2024-07-03 04:20:03.183242: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 04:20:03.183476: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 04:20:03.379070: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
X = np.array(list1)

# b2 = np.array(list2)
# bb2 = b2.reshape(b2.shape[0],)
# y2 = np.int32(bb2)

b10 = np.array(list10)
bb10 = b10.reshape(b10.shape[0],)
y10 = np.int32(bb10)

In [5]:
X.shape

(1778030, 12)

In [ ]:
# sorted(Counter(y2).items())

In [6]:
sorted(Counter(y10).items())

[(0, 1553132),
 (1, 1874),
 (2, 1630),
 (3, 11449),
 (4, 31167),
 (5, 16972),
 (6, 150836),
 (7, 9791),
 (8, 1057),
 (9, 122)]

In [7]:
#View the distribution of data classes (data distribution without imbalanced processing)
# print(Counter(y2))
print(Counter(y10))

Counter({0: 1553132, 6: 150836, 4: 31167, 5: 16972, 3: 11449, 7: 9791, 1: 1874, 2: 1630, 8: 1057, 9: 122})


### SMOTE-binary-class

In [ ]:
# from imblearn.over_sampling import SMOTE

# smo = SMOTE(random_state = 42)

# X_smo, y_smo = smo.fit_resample(X, y2)
# sorted(Counter(y_smo).items())

In [ ]:
# X_smo.shape[0]

In [ ]:
# yy2 = y_smo.reshape(y_smo.shape[0], 1)
# yy2.shape

In [ ]:
# create_file_if_not_exist("imbalance_processing/", "data_smote_binary_train.npy")
# create_file_if_not_exist("imbalance_processing/", "label_2_smote_train.npy")

# np.save("/content/imbalance_processing/data_smote_binary_train.npy", X_smo)
# np.save("/content/imbalance_processing/label_2_smote_train.npy", yy2)

### SMOTE-multi-class

In [8]:
from imblearn.over_sampling import SMOTE
import time
time_start = time.time()
a = 177803 # I_resample = N / C

smo = SMOTE(sampling_strategy={1:a,2:a,3:a,4:a,5:a,6:a,7:a,8:a,9:a},random_state=42)  

X_smo, y_smo = smo.fit_resample(X, y10)   
print(sorted(Counter(y_smo).items()))

time_end = time.time()
time = time_end - time_start
print("time:",time)

[(0, 1553132), (1, 177803), (2, 177803), (3, 177803), (4, 177803), (5, 177803), (6, 177803), (7, 177803), (8, 177803), (9, 177803)]
time: 208.57021045684814


In [9]:
X_smo.shape[0]

3153359

In [ ]:
# yy10 = y_smo.reshape(y_smo.shape[0], 1)
# yy10.shape

In [ ]:
# create_file_if_not_exist("imbalance_processing/", "data_smote_multi_train.npy")
# create_file_if_not_exist("imbalance_processing/", "label_10_smote_train.npy")

# np.save("/content/imbalance_processing/data_smote_multi_train.npy", X_smo)
# np.save("/content/imbalance_processing/label_10_smote_train.npy", yy10)

# del yy10

### Extract Majority class of data

In [11]:
list0 = []
list1 = []
list2 = []

for i in range(X_smo.shape[0]):
    if y_smo[i] == 0:
        list0.append(X_smo[i])
    else:
        list1.append(X_smo[i])
        list2.append(y_smo[i])

In [12]:
data0 = np.array(list0)
data1 = np.array(list1)
label1 = np.array(list2)

label11 = label1.reshape(label1.shape[0],)

print("Normal class data shape：", data0.shape)
print("Attack class data shape：", data1.shape)
print("Attack class label shape：", label11.shape)

Normal class data shape： (1553132, 12)
Attack class data shape： (1600227, 12)
Attack class label shape： (1600227,)


In [13]:
label11

array([5, 6, 6, ..., 9, 9, 9], dtype=int32)

### Cluster majority data into C (total number of classes)

In [14]:
from sklearn.mixture import GaussianMixture
import time
time_start = time.time()

estimator = GaussianMixture(n_components = 10)
estimator.fit(data0)

time_end = time.time()
time = time_end - time_start
print("time:",time)

label_pred = estimator.predict(data0)

time: 130.87832951545715


In [15]:
sorted(Counter(label_pred).items())

[(0, 194479),
 (1, 134590),
 (2, 366233),
 (3, 157219),
 (4, 38662),
 (5, 346081),
 (6, 196772),
 (7, 26),
 (8, 31751),
 (9, 87319)]

### Select a certain amount of data from each cluster to form a new majority data

In [16]:
c0 = []
c1 = []
c2 = []
c3 = []
c4 = []
c5 = []
c6 = []
c7 = []
c8 = []
c9 = []

s0=s1=s2=s3=s4=s5=s6=s7=s8=s9=0

for i in range(data0.shape[0]):
    if label_pred[i] == 0:
        c0.append(data0[i])
        s0=s0+1
    elif label_pred[i] == 1:
        c1.append(data0[i])
        s1=s1+1
    elif label_pred[i] == 2:
        c2.append(data0[i])
        s2=s2+1
    elif label_pred[i] == 3:
        c3.append(data0[i])
        s3=s3+1
    elif label_pred[i] == 4:
        c4.append(data0[i])
        s4=s4+1
    elif label_pred[i] == 5:
        c5.append(data0[i])
        s5=s5+1
    elif label_pred[i] == 6:
        c6.append(data0[i])
        s6=s6+1
    elif label_pred[i] == 7:
        c7.append(data0[i])
        s7=s7+1
    elif label_pred[i] == 8:
        c8.append(data0[i])
        s8=s8+1
    elif label_pred[i] == 9:
        c9.append(data0[i])
        s9=s9+1

In [17]:
a = 19753 # (I_resample - c7) / (C - 1) = (177803 - 26) / 9

In [18]:
del c0[a:len(c0)]
del c1[a:len(c1)]
del c2[a:len(c2)]
del c3[a:len(c3)]
del c4[a:len(c4)]
del c5[a:len(c5)]
del c6[a:len(c6)]
# del c7[a:len(c7)]
del c8[a:len(c8)]
del c9[a:len(c9)]

In [19]:
c00 = np.array(c0)
c11 = np.array(c1)
c22 = np.array(c2)
c33 = np.array(c3)
c44 = np.array(c4)
c55 = np.array(c5)
c66 = np.array(c6)
c77 = np.array(c7)
c88 = np.array(c8)
c99 = np.array(c9)

In [20]:
q = np.concatenate((c00,c11,c22,c33,c44,c55,c66,c77,c88,c99), axis=0)
q.shape

(177803, 12)

In [21]:
label_zc = np.zeros((q.shape[0],), dtype=int)
label_zc.shape

(177803,)

In [22]:
data_end = np.concatenate((q,data1), axis=0)
label_end = np.concatenate((label_zc,label1), axis=0)

In [23]:
sorted(Counter(label_end).items())

[(0, 177803),
 (1, 177803),
 (2, 177803),
 (3, 177803),
 (4, 177803),
 (5, 177803),
 (6, 177803),
 (7, 177803),
 (8, 177803),
 (9, 177803)]

In [24]:
label_end = label_end.reshape(label_end.shape[0],1)

In [26]:
create_file_if_not_exist("imbalance_processing/", "SGM_data_train.npy")
create_file_if_not_exist("imbalance_processing/", "SGM_label10_train.npy")

np.save("/kaggle/working/imbalance_processing/SGM_data_train.npy", data_end)
np.save("/kaggle/working/imbalance_processing/SGM_label10_train.npy", label_end)